# <font color=green>APR - Travaux Pratiques n°7.</font>

> Ce sujet est en lien avec le quatrième chapitre du cours, et concerne la programmation CUDA. Les mêmes commentaires que ceux des derniers TP s’appliquent ici aussi. 
>
> Dans cette séance, l’objectif est de pratiquer la programmation CUDA, autour des patrons en temps constants (sur machine PRAM) et du patron REDUCE. *Dans tous les exercices, l’idée reste la même* : **transformer une image en lui donnant un effet par bloc de taille 32 par 32**. La couleur des pixels de chaque bloc sera unique par bloc dans l’image résultat. Elle résulte d’un calcul assez simple : la moyenne des pixels du même bloc de l’image source. Vous trouverez la version CPU dans la classe src/Exercise1/ImageBlockEffect.h.
>
> En résumé, nous allons jouer avec l’implantation de la réduction. 
>
> **<font color=pink>N'oubliez d'exécuter les deux premières cellules de code afin d'installer l'extension CUDA et de vérifier son bon fonctionnement.</font>**

## <font color=green>Installation du sous-sytème</font>

In [ ]:
# vérifions l'installation du SDK Cuda ...
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Installons l'extension CUDA (n'hésitez par à aller sur la page GitHub ...)
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git &> /dev/null
%load_ext nvcc_plugin
# Installons gdown pour charger fichier depuis Google Drive
!pip install --upgrade --no-cache-dir gdown &> /dev/null
# Installons g++-8
!sudo apt install g++-8 &> /dev/null
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 700 --slave /usr/bin/g++ g++ /usr/bin/g++-7
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 800 --slave /usr/bin/g++ g++ /usr/bin/g++-8
# importation Python pour charger/afficher des images
from google.colab.patches import cv2_imshow
import cv2
def afficher(file, width):
  img = cv2.imread(file)
  height = int(img.shape[0] * width / float(img.shape[1]))
  resized = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA) 
  cv2_imshow(resized)

---
# <font color=green>TP</font>
> L'installation s'est bien déroulée ?Parfait, maintenant au travail !
>
> En premier, il faut charger le TP7 depuis le drive Google ... Vous pouvez charger ce fichier (*i.e.* le premier, le second contient des images) sur votre ordinateur pour étudiez les interfaces, bien que la plupart soient dans le cours ...


In [ ]:
# Chargeons le TP7
!gdown https://drive.google.com/uc?id=1wIIk5BAYwB2WjEeZgEjCa3dVmah2VbgO
!gdown https://drive.google.com/uc?id=1FrHh5Pr2KlirwH6NBOrkfl-COSa9biTq
!unzip -oqq TP7.zip
!unzip -oqq Images-TP6.zip # mêmes images que le TP6 ;-)


>
> Le code du TP est dans le répertoire TP7. Vous pouvez le vérifier dans une cellule en tapant " !ls TP7" par exemple ...
>
> Nous démarrons avec l'exercice 1. 
---
## <font color=green>Exercice 1</font>
>
> **Ce premier exercice se veut très simple : il s’agit de calculer la couleur de chaque bloc de l’image résultat. Pour cela, il faut reprendre la version de la réduction vue en cours, et accessible dans utils/OPP/OPP_cuda_reduce.cuh. Cela passe par un noyau qui doit :**
> *  **Charger le bloc en mémoire partagée.**
> *  **Appliquer la réduction par bloc en reprenant la fonction du cours.**
> *	 **En déduire la couleur du bloc et l’écrire dans l’image résultat (en remplissant le bloc, donc).**
>
> **Ici, chaque bloc de l’image est traité par un bloc de threads de taille 1024 (soit 32 warps). Cette première version est une application directe du cours (vous pouvez quand même remplacer le foncteur par l’addition classique).**
>
> **Attention : l’image est aplatie, id est les blocs de pixels sont aux adresses 1024×i.**
>
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP7/student/exo1/student.cu 
#include <iostream>
#include <exo1/student.h>
#include <OPP_cuda.cuh>

using uchar = unsigned char;

namespace 
{
	// L'idée est de recopier le code du cours (qui est dans utils/OPP_cuda_reduce.cuh)
	
	// Mais, la différence est qu'ici la réduction se fait par bloc de 1024 pixels,
	// un peu comme une réduction par segment, mais avec des segments implicites (chaque bloc est un segment).

	// Donc, il y a uniquement des réductions par blocs de pixels en utilisant threadIdx.x.

	// Un bloc de pixel va correspondre dans ce premier exercice à un bloc de threads (1024 dans les deux cas)

	//
	__device__ 
	__forceinline__
	void loadSharedMemory(float const*const data) 
	{
		// La mémoire partagée contient des FLOAT, donc il faut changer de type
		float*const shared = OPP::CUDA::getSharedMemory<float>();
		// position dans le tableau
		const auto tid = threadIdx.x + blockIdx.x * blockDim.x;
		// position dans le bloc/segment
		shared[threadIdx.x] = data[tid]; 
		__syncthreads();
	}

	//
	__device__ 
	__forceinline__
	void reduceJumpingStep(const int jump)
	{
		// TODO
	}

	//
	__device__
	__forceinline__
	float reducePerBlock(
		float const*const source
	) {
		// TODO
	}

	//
	__device__
	__forceinline__
	void fillBlock(
		const float color, 
		float*const result
	) {
		const unsigned tid = threadIdx.x + blockIdx.x * blockDim.x;
		result[tid] = color;
	}

	//
	__global__
	void blockEffectKernel( 
		float const*const source, 
		float *const result
	) {
		const float sumInBlock = reducePerBlock(source);
		fillBlock(sumInBlock, result);
	}
}

// Cette fonction sera appelée trois fois pour une image donnée, car l'image est séparée en trois tableaux,
// l'un pour le rouge, l'autre pour le vert et enfin le dernier pour le bleu. 
// Cela simplifie le code et réduit la pression sur les registres ;-)
void StudentWorkImpl::run_blockEffect(
	OPP::CUDA::DeviceBuffer<float>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_result
) {
	const auto size = dev_source.getNbElements();
	const auto nbWarps = 32;
	dim3 threads(32*nbWarps);
	dim3 blocks(( size + threads.x-1 ) / threads.x);
	const size_t sizeSharedMemory(threads.x*sizeof(float));
	::blockEffectKernel<<<blocks, threads, sizeSharedMemory>>>(
		dev_source.getDevicePointer(),
		dev_result.getDevicePointer()
	);
}

In [ ]:
# exécutez cette cellule pour consulter le code du reduce "classique" ;-)
!cat TP7/utils/OPP/OPP_cuda_reduce.cuh
#!cat TP7/CMake*

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP7 ; sh ./build.sh exo1

> ### <font color=green>Exécution</font>
> Exécutez la cellule suivante pour exécuter le code ...
>
> Pour le rapport, jouez avec la taille (pour les statistiques, cela signifie prendre des tailles importantes ...). 

In [ ]:
# launch student work
!./TP7/linux/exo1 -i=./Images/Flower_600x450.ppm
# display input image
print("\nInput image is:")
afficher(file="./Images/Flower_600x450.ppm", width=600)
# display expected result
print("\nExpected result is:")
afficher(file="./Images/Flower_600x450_block_reference.ppm", width=600)
# display result
print("\nYour result is:")
afficher(file="Images/Flower_600x450_block_student.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work
!./TP7/linux/exo1 -i=./Images/Raffael_012.ppm
# display result
afficher("Images/Raffael_012_block_student.ppm", 300)

In [ ]:
# launch student work
!./TP7/linux/exo1 -i=./Images/asphalt-highway.ppm
# display result
afficher("Images/asphalt-highway_block_student.ppm", 600)

## <font color=green>Exercice 2</font>

> **Vous avez probablement noté que la version GPU n’est pas beaucoup plus rapide que la version CPU. Comme dit René, ce n’est pas taupe. Plusieurs raisons expliquent cela, nous allons essayer de les traiter l’une après l’autre. Dans cet exercice, l’idée est de réduire la <font color=pink>pression sur les registres</font>.**
>
> **Vous vous souvenez qu’un SMP possède un nombre fixe de registres (par exemple 64k ou 128k). Ces registres sont utilisés pour stocker les variables auto (i.e. déclarées dans vos fonctions), ainsi que les mémoires partagée et constante (les paramètres des fonctions sont en mémoire globale, en revanche). En utilisant la mémoire partagée pour la réduction, notre première version a donc requis un grand nombre de registres par bloc. Calculons cela : il vous a fallu 1024 registres pour une étape de réduction (voire 3 fois plus si jamais vous avez chargé toutes les couleurs en même temps !). Vous avez lancé 32 warps par bloc, donc 1024 threads. En supposant que chaque thread nécessite 20 registres (ce qui est très peu, en fait), alors un bloc requière 1024+20*1024 registres, soit 21k ; un SMP avec 64k registres peut alors faire tourner 3 blocs en parallèle, et pas un de plus. En supposant 32 registres maximum par thread, alors vous aurez un seul bloc par SMP ! Pas terrible pour l’efficacité !**
>
> **Ainsi, comme pour la version vue en cours, nous devons réduire le nombre de warps par bloc pour optimiser le parallélisme. <font color=pink>Utilisez-en moins dans cet exercice</font>. Pour cela il faudra modifier le fonctionnement de la réduction (en clair : dans la partie chargement il faut qu’un thread charge plusieurs valeurs, tout en respectant la propriété d’associativité ; autant ajouter ces valeurs dans un registre privé, puis charger cette somme dans la mémoire partagée …).**
>
> **<font color=pink>Attention</font> : un bloc de threads traite toujours 1024 pixels !**
> 
> **Indiquez dans le rapport ce qui fonctionne le mieux sur votre GPU (indiquez lequel) en termes de taille d’un bloc (faites des essais avec l’option -w, mais attention : le schéma de calcul du reduce ne fonctionne qu’avec un nombre de warps qui est une puissance de 2 !).**
>
> **<font color=pink>NB : pensez parallèle ! Un algorithme séquentiel est inadaptable ...</font>**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%cuda --name ../TP7/student/exo2/student.cu
#include <iostream>
#include <exo2/student.h>
#include <OPP_cuda.cuh>

using uchar = unsigned char;

namespace 
{
	// L'opération est associative (enfin, en toute généralité), et donc les permutations de valeurs sont interdites.
	// Seul les changements de parenthèses sont autorisées ...
	// Donc il y a deux solutions :
	// - La plus simple est d'effectuer plusieurs réductions successives par blocs
	// - La plus difficile mais efficace, et de grouper les valeurs consécutives par thread.
	// Avec cette seconde, le premier thread (0) va traiter des valeurs consécutives. Le thread suivant aussi, etc.
	// En supposant par exemple que chaque thread traite 4 valeurs, alors les 4 premiers pixels du blocs sont utilisés par
	// le thread 0, le 4 suivant par le thread 1, etc. jusqu'au thread 255 ;-)
	// NB : on suppose que le nombre de warps est une puissance de 2 (et donc divise 1024)
	template<int NB_WARPS>
	__device__ 
	__forceinline__
	void loadSharedMemoryAssociate(float const*const data) 
	{
		float*const shared = OPP::CUDA::getSharedMemory<float>();

		const auto globalOffset = 1024 * blockIdx.x;
		const auto localThreadId = threadIdx.x;
		const unsigned nbPixelsPerThread = (1024 + 32*NB_WARPS - 1) / (32*NB_WARPS);

		float sumPerThread = 0.f;

		for(unsigned i=0; i<nbPixelsPerThread; ++i) 
		{
			// indice du pixel à traiter
			const auto pixelIdInBlock = nbPixelsPerThread * localThreadId + i;
			
			// TODO
		}
		shared[localThreadId] = sumPerThread;
		__syncthreads();
	}


	// idem exo1, sauf test de débordement
	template<int NB_WARPS>
	__device__ 
	__forceinline__
	void reduceJumpingStep(const int jump)
	{
		// TODO 
	}


	// on ne changera ici que le nombre d'itérations (10 avant, ici moins)
	template<int NB_WARPS>
	__device__
	__forceinline__
	float reducePerBlock(
		float const*const source
	) {
		// TODO
	}	
	

	// ressemble beaucoup à l'exo1 ...
	template<int NB_WARPS>
	__device__
	__forceinline__
	void fillBlock(
		const float color, 
		float*const result
	) {
		// calcul de l'offset du bloc : la taille est 1024
		const auto offset = blockIdx.x * 1024;
		// TODO
	}


	// idem exo1 with templates
	template<int NB_WARPS>
	struct EvaluateWarpNumber {
		enum { res = 1 };
	};
	template<>
	struct EvaluateWarpNumber<1> {
		enum { res = 16 };
	};
	template<>
	struct EvaluateWarpNumber<2> {
		enum { res = 8 };
	};
	template<>
	struct EvaluateWarpNumber<4> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<8> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<16> {
		enum { res = 2 };
	};

	// idem exo1
	template<int NB_WARPS=32>
	__global__
	__launch_bounds__(32*NB_WARPS , EvaluateWarpNumber<NB_WARPS>::res)
	void blockEffectKernel( 
		float const*const source, 
		float *const result
	) {
		const float sumInBlock = reducePerBlock<NB_WARPS>(source);
		fillBlock<NB_WARPS>(sumInBlock, result);
	}
}


// idem exo1, sauf la taille d'un bloc de threads (et les templates)
void StudentWorkImpl::run_blockEffect(
	OPP::CUDA::DeviceBuffer<float>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_result,
	const unsigned nbWarps
) {
	// Le nombre de warps est réduit ...
	const auto size = dev_source.getNbElements();
	// Le nombre de threads par bloc dépend du nombre de warps ;-)
	dim3 threads(32 * nbWarps); 
	// Attention : le nombre de blocs est calculer en considérant des traitements de 1024 pixels ! 
	dim3 blocks ((size + 1024-1) / 1024 );
	// le reste est classique
	const size_t sizeSharedMemory(threads.x*sizeof(float));
	switch(nbWarps) {
		case 1:
			::blockEffectKernel<1> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 2:
			::blockEffectKernel<2> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 4:
			::blockEffectKernel<4> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 8:
			::blockEffectKernel<8> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 16:
			::blockEffectKernel<16> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 32:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		default:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
	}
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP7 ; sh ./build.sh exo2
!ls Images

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...
>
> N'hésitez pas à jouer avec le paramètre -w=?? pour tester différents nombres de warps ...

In [ ]:
# launch student work (w is number of warps, you can modify it into 1..8)
!./TP7/linux/exo2 -i=./Images/Flower_600x450.ppm -w=8
# display result
afficher(file="Images/Flower_600x450_block_student.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work
!./TP7/linux/exo2 -i=./Images/Raffael_012.ppm -w=8
# display result
afficher("Images/Raffael_012_block_student.ppm", 300)

In [ ]:
# launch student work
!./TP7/linux/exo2 -i=./Images/asphalt-highway.ppm -w=8
# display result
afficher("Images/asphalt-highway_block_student.ppm", 600)

## <font color=green>Exercice 3</font>

> **Dans l’exercice 2, le respect de la propriété d’associativité a rendu le chargement de la mémoire cache un peu délicat (chargement par bloc pour un même thread). Une version plus simple consiste à charger les données par bloc pour tous les threads : un thread charge la valeur à l’indice correspondant à sa position, puis la valeur de même indice plus le nombre total de threads, puis la valeur à son indice plus le nombre total de threads fois 2, etc. Ce schéma plus efficace (en termes de coalescence) nécessite la propriété de commutativité, qui n’est pas requise dans une réduction classique (pensez au produit de matrices …). Ici, l’opération associative est une simple addition sur des réels, donc commutative. **
>
> **Modifiez dans cet exercice le chargement des données en mémoire cache.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%cuda --name ../TP7/student/exo3/student.cu
#include <iostream>
#include <exo3/student.h>
#include <OPP_cuda.cuh>

using uchar = unsigned char;

namespace 
{
	// Beaucoup de solutions ici sont possibles ...
	// Celle-ci traite le bloc de pixels par morceau. Chaque pixel d'un morceau est traité par un thread.
	// On répète le processus jusqu'à avoir couvert tout le bloc de pixel.
	// NB: il faut un réduction par thread (séquentielle, avec variable privée)
	template<int NB_WARPS>
	__device__ 
	__forceinline__
	void loadSharedMemoryCommutative(float const*const data) 
	{
		float*const shared = OPP::CUDA::getSharedMemory<float>();
		float sum = 0.f;
		const unsigned globalOffset = blockIdx.x * 1024; 
		for(auto tid = threadIdx.x; tid < 1024; tid += 32*NB_WARPS) 
		{
			// TODO
		}
		const auto localThreadId = threadIdx.x;
		shared[localThreadId] = sum;
		__syncthreads();
	}

	// idem exo2
	template<int NB_WARPS>
	__device__ 
	__forceinline__
	void reduceJumpingStep(const int jump)
	{
		// TODO
	}


	// Idem exo2, sauf le nom de la fonction de chargement ;-)
	template<int NB_WARPS>
	__device__
	__forceinline__
	float reducePerBlock(
		float const*const source
	) {
		// TODO
	}	
	

	// idem exo2
	template<int NB_WARPS>
	__device__
	__forceinline__
	void fillBlock(
		const float color, 
		float*const result
	) {
		// TODO
	}


	// idem exo1
	template<int NB_WARPS>
	struct EvaluateWarpNumber {
		enum { res = 1 };
	};
	template<>
	struct EvaluateWarpNumber<1> {
		enum { res = 16 };
	};
	template<>
	struct EvaluateWarpNumber<2> {
		enum { res = 8 };
	};
	template<>
	struct EvaluateWarpNumber<4> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<8> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<16> {
		enum { res = 2 };
	};
	
	template<int NB_WARPS=32>
	__global__
	__launch_bounds__(32*NB_WARPS , EvaluateWarpNumber<NB_WARPS>::res)
	void blockEffectKernel( 
		float const*const source, 
		float *const result
	) {
		const float sumInBlock = reducePerBlock<NB_WARPS>(source);
		fillBlock<NB_WARPS>(sumInBlock, result);
	}
}


void StudentWorkImpl::run_blockEffect(
	OPP::CUDA::DeviceBuffer<float>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_result,
	const unsigned nbWarps
) {
	const auto size = dev_source.getNbElements();

	dim3 threads(32 * nbWarps); 
	dim3 blocks((size +1023) / 1024);
	const size_t sizeSharedMemory(threads.x*sizeof(float));
	switch(nbWarps) {
		case 1:
			::blockEffectKernel<1> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 2:
			::blockEffectKernel<2> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 4:
			::blockEffectKernel<4> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 8:
			::blockEffectKernel<8> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 16:
			::blockEffectKernel<16> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 32:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		default:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
	}
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP7 ; sh ./build.sh exo3

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work (w is number of warps, you can modify it into 1..8)
!./TP7/linux/exo3 -i=./Images/Flower_600x450.ppm -w=1
# display result
afficher(file="Images/Flower_600x450_block_student.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work
!./TP7/linux/exo3 -i=./Images/Raffael_012.ppm -w=1
# display result
afficher("Images/Raffael_012_block_student.ppm", 300)

In [ ]:
# launch student work
!./TP7/linux/exo3 -i=./Images/asphalt-highway.ppm -w=1
# display result
afficher("Images/asphalt-highway_block_student.ppm", 600)

## <font color=green>Exercice 4</font>

> **Il est possible de tirer parti de la propriété de commutativité pour accélérer encore plus la réduction. Modifiez le schéma de calcul de la fonction reduceJumpingStep en ce sens, de façon à avoir moins de warps au travail.**
> *	**Sur une feuille de papier, dessinez le schéma de calcul actuel (avec 4 warps de 4 threads chacun pour simplifier, c’est suffisant pour réfléchir…).** 
> *	**En déduire un second schéma minimisant le nombre de warps au travail, grâce à la commutation de l’addition.**
> *	**Puis codez ce nouveau schéma.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Ctrl-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP7/student/exo4/student.cu
#include <iostream>
#include <exo4/student.h>
#include <OPP_cuda.cuh>

using uchar = unsigned char;

namespace 
{
	// idem exo3
	template<int NB_WARPS>
	__device__ 
	__forceinline__
	void loadSharedMemoryCommutative(float const*const data) 
	{
		// TODO
	}

	// nouvelle version :-)
	__device__ 
	__forceinline__
	void reduceJumpingStep(const int jump)
	{
		// TODO
	}

	// similaire précédente, mais boucle différente (les threads qui travaillent sont en tête ...)
	template<int NB_WARPS>
	__device__
	__forceinline__
	float reducePerBlock(
		float const*const source
	) {
		// TODO
	}

	
	// idem exo3
	template<int NB_WARPS>
	__device__
	__forceinline__
	void fillBlock(
		const float color, 
		float*const result
	) {
		// TODO
	}


	// idem exo1
	template<int NB_WARPS>
	struct EvaluateWarpNumber {
		enum { res = 1 };
	};
	template<>
	struct EvaluateWarpNumber<1> {
		enum { res = 16 };
	};
	template<>
	struct EvaluateWarpNumber<2> {
		enum { res = 8 };
	};
	template<>
	struct EvaluateWarpNumber<4> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<8> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<16> {
		enum { res = 2 };
	};
	template<int NB_WARPS=32>
	__global__
	__launch_bounds__(32*NB_WARPS , EvaluateWarpNumber<NB_WARPS>::res)
	void blockEffectKernel( 
		float const*const source, 
		float *const result
	) {
		const float sumInBlock = reducePerBlock<NB_WARPS>(source);
		fillBlock<NB_WARPS>(sumInBlock, result);
	}
}


// Attention : ici la taille des vecteurs n'est pas toujours un multiple du nombre de threads !
// Il faut donc corriger l'exemple du cours ...
void StudentWorkImpl::run_blockEffect(
	OPP::CUDA::DeviceBuffer<float>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_result,
	const unsigned nbWarps
) {
	const auto size = dev_source.getNbElements();
	dim3 threads( 32 * nbWarps );
	dim3 blocks( (size + 1023) / 1024 );
	const size_t sizeSharedMemory(threads.x*sizeof(float));
	switch(nbWarps) {
		case 1:
			::blockEffectKernel<1> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 2:
			::blockEffectKernel<2> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 4:
			::blockEffectKernel<4> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 8:
			::blockEffectKernel<8> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 16:
			::blockEffectKernel<16> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 32:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		default:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
	}

}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP7 ; sh ./build.sh exo4

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work (w is number of warps, you can modify it into 1..8)
!./TP7/linux/exo4 -i=./Images/Flower_600x450.ppm -w=4
# display result
afficher(file="Images/Flower_600x450_block_student.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work
!./TP7/linux/exo4 -i=./Images/Raffael_012.ppm -w=4
# display result
afficher("Images/Raffael_012_block_student.ppm", 300)

In [ ]:
# launch student work
!./TP7/linux/exo4 -i=./Images/asphalt-highway.ppm -w=4
# display result
afficher("Images/asphalt-highway_block_student.ppm", 600)

## <font color=green>Exercice 5</font>

> **La version précédente est efficace car moins de warps travaillent. Vous avez probablement observé sur votre feuille de papier que les dernières itérations n’utilisent qu’un seul et unique warp : pour ajouter 64 valeurs, puis 32, puis 16, puis 8, puis 4, puis enfin les 2 dernières. Un seul warp pour traiter ces 6 étapes ! Nagerions-nous en plein Tolkien ? Et pourtant, vous continuez à synchroniser les threads, ce qui est inutile au sein du même warp (SIMD oblige !).** 
>
> **Supprimez, pour ces dernières réductions (et uniquement celles-ci) la synchronisation. Attention : la mémoire partagée devra avoir l’attribut volatile … sinon le compilateur va chercher à optimiser avec des variables privées.**
>
> **Refaites l’étude sur le nombre de threads optimal.**
> 
> **Et voilà, vous avez de quoi écrire une nouvelle version de la réduction, pour une fonction associative et commutative …**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP7/student/exo5/student.cu
#include <iostream>
#include <exo5/student.h>
#include <OPP_cuda.cuh>

using uchar = unsigned char;

namespace 
{
	// idem exo3
	template<int NB_WARPS>
	__device__ 
	__forceinline__
	void loadSharedMemoryCommutative(float const*const data) 
	{
		// TODO
	}

	// idem exo4
	__device__ 
	__forceinline__
	void reduceJumpingStep(const int jump)
	{
		// TODO
	}

	// nouvelle fonction !
	__device__ 
	__forceinline__
	void reduceLastWarp()
	{
		// attention au mot clé volatile ... essentiel !
		volatile float*const shared = OPP::CUDA::getSharedMemory<float>();
		const auto tid = threadIdx.x;
		if( tid < 32 ) 
		{
			// TODO
		}
		__syncthreads();
		
	}
	
	// 
	template<int NB_WARPS>
	__device__
	__forceinline__
	float reducePerBlock(
		float const*const source
	) {
		// TODO
	}

	
	// idem exo3
	template<int NB_WARPS>
	__device__
	__forceinline__
	void fillBlock(
		const float color, 
		float*const result
	) {
		// TODO
	}


	// idem exo1
	template<int NB_WARPS>
	struct EvaluateWarpNumber {
		enum { res = 1 };
	};
	template<>
	struct EvaluateWarpNumber<1> {
		enum { res = 16 };
	};
	template<>
	struct EvaluateWarpNumber<2> {
		enum { res = 8 };
	};
	template<>
	struct EvaluateWarpNumber<4> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<8> {
		enum { res = 4 };
	};
	template<>
	struct EvaluateWarpNumber<16> {
		enum { res = 2 };
	};
	template<int NB_WARPS=32>
	__global__
	__launch_bounds__(32*NB_WARPS , EvaluateWarpNumber<NB_WARPS>::res)
	void blockEffectKernel( 
		float const*const source, 
		float *const result
	) {
		const float sumInBlock = reducePerBlock<NB_WARPS>(source);
		fillBlock<NB_WARPS>(sumInBlock, result);
	}
}


// Attention : ici la taille des vecteurs n'est pas toujours un multiple du nombre de threads !
// Il faut donc corriger l'exemple du cours ...
void StudentWorkImpl::run_blockEffect(
	OPP::CUDA::DeviceBuffer<float>& dev_source,
	OPP::CUDA::DeviceBuffer<float>& dev_result,
	const unsigned nbWarps
) {
	const auto size = dev_source.getNbElements();
	dim3 threads(32*nbWarps);
	dim3 blocks((size + 1023) / 1024);
	const size_t sizeSharedMemory(threads.x*sizeof(float));

	switch(nbWarps) {
		case 1:
			::blockEffectKernel<1> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 2:
			::blockEffectKernel<2> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 4:
			::blockEffectKernel<4> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 8:
			::blockEffectKernel<8> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 16:
			::blockEffectKernel<16> <<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		case 32:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
		default:
			::blockEffectKernel<32><<<blocks, threads, sizeSharedMemory>>>(
				dev_source.getDevicePointer(),
				dev_result.getDevicePointer()
			);
			return;
	}
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP7 ; sh ./build.sh exo5

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...
>
> Pour le rapport, jouez avec le nombre de warps (pour les statistiques). 

In [ ]:
# launch student work (w is number of warps, you can modify it into 1..8)
!./TP7/linux/exo5 -i=./Images/Flower_600x450.ppm -w=4
# display result
afficher(file="Images/Flower_600x450_block_student.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work
!./TP7/linux/exo5 -i=./Images/Raffael_012.ppm -w=4
# display result
afficher("Images/Raffael_012_block_student.ppm", 300)

In [ ]:
# launch student work
!./TP7/linux/exo5 -i=./Images/asphalt-highway.ppm -w=4
# display result
afficher("Images/asphalt-highway_block_student.ppm", 600)

# <font color=green>That's all, folks!</font>